In [106]:
import pandas as pd
from requests_html import HTMLSession
import geocoder

In [57]:
session = HTMLSession()
r = session.get("https://incidentreports.uchicago.edu/incidentReportArchive.php?startDate=03%2F01%2F2024&endDate=03%2F10%2F2024")
num_pages = int(r.html.search("1 / {}")[0])

In [129]:
df = pd.DataFrame()
for i in range(num_pages):
    link = f"https://incidentreports.uchicago.edu/incidentReportArchive.php?startDate=03%2F01%2F2024&endDate=03%2F10%2F2024&offset={i*5}"
    table = pd.read_html(link)[0]
    df = pd.concat([df, table], ignore_index = True)

In [130]:
df["Location"] = df["Location"].str.replace(" \\(.*", "", regex = True)
df = df[df.Incident.str.match('Theft|Robbery|Battery|Assault')]
df["Location"] = df["Location"] + ", Chicago, IL"
df["Location"] = df["Location"].apply(lambda add: geocoder.osm(add).latlng)
crime_table = df.filter(items = ["Incident", "Location", "Reported"])